In [1]:
import chromadb
from chromadb.config import Settings
from pathlib import Path

# embeddings_dir = Path().parent.parent.resolve() / "embeddings"
embeddings_dir = "../app/embeddings"

In [2]:

client = chromadb.PersistentClient(
    path=str(embeddings_dir),
    settings=Settings(
        anonymized_telemetry=False,
        is_persistent=True
    )
)


In [3]:
client.list_collections()

[Collection(name=template_embeddings)]

In [4]:
# get number of documents in the collection
client.get_collection("template_embeddings").count()

208

In [5]:
template_id='midwit'

collection = client.get_or_create_collection(
            name="template_embeddings",
            metadata={"hnsw:space": "cosine"}  # Use cosine similarity for OpenAI embeddings
        )

# Get existing document and embedding
result = collection.get(
    ids=[template_id],
    include=['embeddings', 'documents', 'metadatas']
)

Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
Add of existing embedding ID: config
A

In [17]:
result

{'ids': ['midwit'],
 'embeddings': array([[-0.00754775, -0.05613894, -0.04741969, ..., -0.02111039,
         -0.02993455,  0.0119948 ]], shape=(1, 1024)),
 'documents': ['Name: Midwit Keywords: IQ Bell Curve / Midwit Source: https://knowyourmeme.com/memes/iq-bell-curve-midwit Example: if it\'s good I do it NOOOOO you have to have complex reasons for what you\'re doing. what is the point. what is your goal. if it\'s good I do it Description: IQ Bell Curve / Midwit | Know Your Meme Hauu! You must login or signup first! Already a memeber? Login Now! Don\'t have an account? Sign up Now! Meme Encyclopedia All Cultures Events People Sites Subcultures Videos Submit → Images Trending Most Commented Most Favorited High Score Low Score Most Viewed All Templates Upload an Image Articles News Explainers Collections Interviews Meme Insider Magazine More Discord Newsletter Episodes Forums Meme Store Login Abstract user icon Memes Categories Trending News Images Trending Images Videos Trending Videos

In [15]:
import requests
import json
from urllib.parse import quote
import argparse
from typing import Optional
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def search_templates(query: str, n_results: Optional[int] = 5, base_url: str = "http://localhost:5001") -> None:
    """Search for meme templates using the API."""
    # URL encode the query
    encoded_query = quote(query)
    
    # Construct URL with parameters
    url = f"{base_url}/search?q={encoded_query}"
    if n_results:
        url += f"&n={n_results}"
        
    try:
        # Make the request
        logger.info(f"Searching for: {query}")
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse and display results
        results = response.json()["results"]
        logger.info(f"Found {len(results)} results:")
        
        # for i, result in enumerate(results, 1):
        #     print(f"\n{i}. {result['metadata']['name']} (ID: {result['template_id']})")
        #     print(f"   Similarity score: {1 - result['distance']:.3f}")  # Convert distance to similarity
        #     print(f"   Text zones: {result['text_zones']}")
        #     if result['config'].get('keywords'):
        #         print(f"   Keywords: {', '.join(result['config']['keywords'])}")
        #     if result['config'].get('source'):
        #         print(f"   Source: {result['config']['source']}")
        return results
                
    except requests.exceptions.RequestException as e:
        logger.error(f"Error making request: {e}")
    except KeyError as e:
        logger.error(f"Error parsing response: {e}")
        logger.debug(f"Response content: {response.text}")
    except Exception as e:
        logger.error(f"Unexpected error: {e}")


In [16]:
res = search_templates("gambling on memecoins cause fuck it")
res


INFO:__main__:Searching for: gambling on memecoins cause fuck it
INFO:__main__:Found 5 results:


[{'template_id': 'stonks',
  'metadata': {'name': 'Stonks', 'template_id': 'stonks', 'text_zones': 2},
  'config': {'example': ['', 'stonks'],
   'keywords': [None],
   'name': 'Stonks',
   'overlay': [{'angle': 0.0,
     'center_x': 0.5,
     'center_y': 0.5,
     'scale': 0.25}],
   'scraped_content': 'Stonks | Know Your Meme Yo! You must login or signup first! Already a memeber? Login Now! Don\'t have an account? Sign up Now! Meme Encyclopedia All Cultures Events People Sites Subcultures Videos Submit → Images Trending Most Commented Most Favorited High Score Low Score Most Viewed All Templates Upload an Image Articles News Explainers Collections Interviews Meme Insider Magazine More Discord Newsletter Episodes Forums Meme Store Login Abstract user icon Memes Categories Trending News Images Trending Images Videos Trending Videos Editorials Meme Insider Magazine Newsletter Policies and Guidelines Meme Store Old site Login Know Your Meme is the property of Literally Media ©2024 Litera

In [17]:
[i['template_id'] for i in res]

['stonks', 'cheems', 'bender', 'gone', 'elmo']

In [1]:
from scripts.search_templates import search_templates_notebook 

# search_templates_notebook("gambling on memecoins cause fuck it")

ModuleNotFoundError: No module named 'scripts'

In [8]:
import httpx
from bs4 import BeautifulSoup


async def scrape_webpage(url: str) -> dict:
    """Scrape text content and metadata from a webpage."""
    try:
        # Configure client with redirect following and longer timeout
        async with httpx.AsyncClient(follow_redirects=True, timeout=30.0) as client:
            response = await client.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract added_at datetime by finding the "Added" text and then its associated timeago
            added_at = None
            footer_paragraphs = soup.find_all('p')
            for p in footer_paragraphs:
                if p.get_text().strip().startswith('Added'):
                    timeago = p.find('abbr', class_='timeago')
                    if timeago:
                        added_at = timeago.get('title')
                        break
            
            # Find the main entry body
            entry_body = soup.find('div', {'class': 'c', 'id': 'entry_body'})
            if not entry_body:
                logger.warning("Could not find entry body div")
                return None
                
            # Extract left aside content
            aside_content = ""
            aside = entry_body.find('aside', {'class': 'left'})
            if aside:
                # Remove script and style elements from aside
                for script in aside(['script', 'style']):
                    script.decompose()
                aside_content = aside.get_text(strip=True)
                # Remove the aside from entry_body to not duplicate content
                aside.decompose()
            
            # Remove remaining script and style elements from entry_body
            for script in entry_body(['script', 'style']):
                script.decompose()
                
            # Get text and clean it up
            main_text = entry_body.get_text()
            lines = (line.strip() for line in main_text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            main_text = ' '.join(chunk for chunk in chunks if chunk)
            
            return {
                "text": main_text,
                "aside_content": aside_content,
                "added_at": added_at
            }
    except Exception as e:
        logger.error(f"Failed to scrape {url}: {str(e)}")
        return None
res = await scrape_webpage("https://knowyourmeme.com/memes/but-we-are-also-roommates")
res


{'text': '[Warning: Contains spoilers for the final Team Fortress 2 comic] About "But We Are Also Roommates" is a redraw meme inspired by a panel from the final Team Fortress 2 comic, where Soldier and his wife Zhanna have a back-and-forth about the nature of their relationship. The meme replaces Soldier and Zhanna from the comic panel with different characters. Origin The final Team Fortress 2 comic titled The Days Have Worn Away[1] was released on December 20th, 2024. The comic contained a scene where Soldier and his wife Zhanna, break into the Badlands Correctional Institute to retrieve Merasmus, who was being held prisoner. Soldier introduces his wife to Merasmus, referring to her as his "roommate," prompting Zhanna correct him by saying she\'s his wife. Soldier then responds "But we are also roommates." The panel containing this dialogue was posted to Twitter / X[2] by @ginfantasy on December 20th, 2024, gathering over 118,000 likes and over 5,800 reposts in less than a week. Spre

In [6]:
from app.embeddings import get_embedding, date_to_timestamp
from dotenv import load_dotenv
from datetime import datetime
import logging

# Load environment variables
load_dotenv('../.env')

logger = logging.getLogger(__name__)

In [11]:
# query = "gambling on memecoins cause fuck it"
query = "meme"
n_results = 5
from_date = "2020-01-01"
to_date = None # "2024-01-31"

# Get embedding for query
query_embedding = get_embedding(query)
if not query_embedding:
    logger.error("Failed to get embedding for query")
    # return []
    
# Prepare where clause for date filtering
where = {}
if from_date:
    # Convert YYYY-MM-DD to ISO format for comparison
    from_epoch = date_to_timestamp(from_date)
    where["added_at_ts"] = {"$gte": from_epoch}
if to_date:
    # Convert YYYY-MM-DD to ISO format for comparison
    to_epoch = date_to_timestamp(to_date)
    if "added_at_ts" in where:
        where["added_at_ts"]["$lte"] = to_epoch
    else:
        where["added_at_ts"] = {"$lte": to_epoch}
    
logger.debug(f"Using where clause: {where}")
    
# Search ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=n_results,
    where=where if where else None,
    include=['metadatas', 'documents', 'distances']
)

results


{'ids': [['panik-kalm-panik', 'kramer', 'stop', 'mouth', 'pool']],
 'embeddings': None,
 'documents': [['Name: Panik Kalm Panik Source: https://knowyourmeme.com/memes/panik-kalm Example: You hear a sound downstairs It\'s just a cat You don\'t have a cat Description: Panik Kalm | Know Your Meme Howdy! You must login or signup first! Already a memeber? Login Now! Don\'t have an account? Sign up Now! Meme Encyclopedia All Cultures Events People Sites Subcultures Videos Submit → Images Trending Most Commented Most Favorited High Score Low Score Most Viewed All Templates Upload an Image Articles News Explainers Collections Interviews Meme Insider Magazine More Discord Newsletter Episodes Forums Meme Store Login Abstract user icon Memes Categories Trending News Images Trending Images Videos Trending Videos Editorials Meme Insider Magazine Newsletter Policies and Guidelines Meme Store Old site Login Know Your Meme is the property of Literally Media ©2024 Literally Media. All Rights Reserved. 